In [5]:
import re
import nltk
import emoji
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from itertools import groupby
import spacy
nlp = spacy.load("it_core_news_sm")

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gabrielebenanti/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gabrielebenanti/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Data Preprocessing


In [6]:
correct_words = ['alcool', 'cool', 'preesame', ]

In [7]:
def clean_disguised_bad_words(text):
    text = " " + text + " "
    print(text)
    text = re.sub(r' c[glo.x*@%#$^]+i ', ' coglioni ', text)
    text = re.sub(r' c[glo.x*@%#$^]+e ', ' coglione ', text)
    text = re.sub(r' c[az.x*@%#$^]+o ', ' cazzo ', text) 
    text = re.sub(r' c[az.x*@%#$^]+ro ', ' cazzaro ', text) 
    text = re.sub(r' c[az.x*@%#$^]+i ', ' cazzi ', text) 
    text = re.sub(r' m[erd.x*@%#$^]+a ', ' merda ', text) 
    text = re.sub(r' m[erd.x*@%#$^]+@ ', ' merda ', text)
    text = re.sub(r' m[erd.x*@%#$^]+e ', ' merde ', text) 
    text = re.sub(r' c[.x*@%#$^]+lo ', ' culo ', text) 
    text = re.sub(r' p[tu.x*@%#$^]+a ', ' puttana ', text)
    text = re.sub(r' p[tu.x*@%#$^]+e ', ' puttane ', text)
    text = re.sub(r' t[.x*@%#$^]+a ', ' troia ', text)
    text = re.sub(r' t[.x*@%#$^]+e ', ' troie ', text)
    text = re.sub(r' s[.x*@%#$^]+o ', ' stronzo ', text)
    text = re.sub(r' s[.x*@%#$^]+i ', ' stronzi ', text)
    return text

def deleteDuplicate (riga):
    parole = riga.split()
    stringa = ""
    for a in parole:
        parola = a
        a = [list(g) for k, g in groupby(a)]    
        vocali = ['a','e','i','o','u','y']
        
        for idx,_ in enumerate(a):
            if idx == 0:
                stringa += a[idx][0] 
            elif idx == len(a)-1:
                stringa += a[idx][0]
            elif a[idx][0] in vocali and (parola not in correct_words):
                stringa += a[idx][0]
            elif len(a[idx]) == 1:
                stringa += a[idx][0]
            elif len (a[idx]) >= 2:
                stringa += a[idx][0]
                stringa += a[idx][1]
        stringa =  stringa + " "     
    return(stringa)

def replace_verbs_with_root(sentence):
     # Dictionary to handle special cases manually
    special_cases = {
        "c'è": "essere",
        "c'erano": "essere",
        "c'era": "essere",
        "ci sono": "essere"
    }

    # Replace special cases in the sentence
    for word, root in special_cases.items():
        sentence = sentence.replace(word, root)
    
    return sentence

def preprocess_tweet(tweet):
    # Remove newlines and carriage returns
    tweet = tweet.replace('\n', ' ').replace('\r', ' ').lower()

    # Remove incorrect bad word 
    tweet = clean_disguised_bad_words(tweet)
    
    # Remove duplicates
    tweet = deleteDuplicate(tweet)

    # Substitute root verbs
    tweet = replace_verbs_with_root(tweet)

    # Substitute URLs with <URL>
    tweet = re.sub(r'\[URL\]', '<URL>', tweet)

    # Substitute mentions with <MENTION>
    tweet = re.sub(r'@\w+', '<MENTION>', tweet)

    # Substitute hashtags with <HASHTAG>
    tweet = re.sub(r'#\w+', '<HASHTAG>', tweet)

    # Replace emojis with text
    tweet = emoji.demojize(tweet, language='it')

    # Tokenization
    tokens = nltk.word_tokenize(tweet)

    # Remove stop words (Italian)
    stop_words = set(stopwords.words('italian'))
    filtered_tokens = " ".join(word for word in tokens if word not in stop_words)

    """ # Stemming using Italian stemmer
    stemmer = SnowballStemmer('italian')
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens] """

    # Lemmatize using spaCy
    doc = nlp(filtered_tokens)
    lemmatize_tokens = [token.lemma_ if token.lemma_ != '-PRON-' else token.text for token in doc]    

    # Remove special characters and numbers, preserve placeholders
    processed_tweet = " ".join(re.sub(r'[^a-zA-Z\<\>]', ' ', token) for token in lemmatize_tokens)

    return processed_tweet

In [8]:

# Example Italian tweet with placeholders
tweet = "Ma davvero 📺 #SalviniMerda49Milioni  alcool c'è che l'ha ciaaaooo c@lo ca**o m*rd@ una raccolta firme per #NoCoprifuocoDeiMieiCoglioni riuscirà a far cambiare idea al #governodeipeggiori? Povero illuso del cazzo"

# Preprocess the tweet
processed_tweet = preprocess_tweet(tweet)

print("Processed Tweet:", processed_tweet)

 ma davvero 📺 #salvinimerda49milioni  alcool c'è che l'ha ciaaaooo c@lo ca**o m*rd@ una raccolta firme per #nocoprifuocodeimieicoglioni riuscirà a far cambiare idea al #governodeipeggiori? povero illuso del cazzo 
Processed Tweet: davvero   televisore   < HASHTAG > alcool essere lo avere ciao culo cazzo merda raccoltare firma < HASHTAG > riuscire fare cambiare idea < HASHTAG >   povero illuso cazzo
